## 1. Fetch the data from the first source, but these data might be abandoned.

Download videos from a playlist. The 14th link fails with some error from YouTube.

In [ ]:
from pytube import YouTube, Playlist

playlist_url = 'https://www.youtube.com/playlist?list=PLoCMsyE1cvdVnCgHk43vRy7PVTVWJ6WVR'

playlist = Playlist(playlist_url)

video_urls = playlist.video_urls

for video_url in list(video_urls)[13:14]:

    url = video_url

    yt = YouTube(url)

    streams = yt.streams
    streams.get_highest_resolution().download()

Download the textbook and PPTs.

In [ ]:
from bs4 import BeautifulSoup
import requests
import os

url = "http://www.mmds.org/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table')

links = table.find_all('a', text=["PPT", "PDF", "HTML"])

for link in links:
    if link['href'][:4] == 'http':
        path = './textbook'
        file_link = link['href']
    else:
        if link['href'][-3:] == 'pdf':
            path = './slides_PDF'
        else:
            path = './slides_PPT'
        file_link = 'http://www.mmds.org/' + link['href'][1:]
        
    file = requests.get(file_link)
    
    file_name = os.path.basename(file_link)
    
    file_path = os.path.join(path, file_name)
    
    folder_path = os.path.dirname(file_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(file_path, 'wb') as f:
        f.write(file.content)

Use downsub.com to download the subtitles(.srt file) of YouTube videos above.

In [ ]:
from pytube import YouTube, Playlist
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time

playlist_url = 'https://www.youtube.com/playlist?list=PLoCMsyE1cvdVnCgHk43vRy7PVTVWJ6WVR'

playlist = Playlist(playlist_url)

video_urls = playlist.video_urls

driver = webdriver.Chrome()
driver.maximize_window() 

driver.get("https://downsub.com/")

for video_url in list(video_urls):

    input_blk = driver.find_element(By.XPATH, '/html/body/div[1]/div/main/div/div[1]/div/div[2]/form/div/div[1]/div[1]/div[1]/input')#//*[@id="input-34"]
    for i in range(10):
        input_blk.send_keys(Keys.CONTROL, "a")
        input_blk.send_keys(Keys.DELETE)
        time.sleep(0.1)
    input_blk.send_keys(video_url)
    time.sleep(2)
    
    submit_button = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div[1]/div/div[2]/form/div/div[2]/button/span')
    submit_button.click()
    time.sleep(10)

    download_button = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/button[1]/span/button/span')
    download_button.click()
    time.sleep(2)

driver.quit()

## 2. Fetch the data from the second source, http://www.mmds.org/.

Get the YouTube video links from course catalog webpage.

In [1]:
from bs4 import BeautifulSoup
import requests
import os

url = "http://www.mmds.org/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table')

a_tags = table.find_all('a')

yt_links=[]
for a_tag in a_tags:
    if a_tag.text not in ["PPT", "PDF", "HTML", ""]:
        yt_links.append(a_tag['href'])

The fetched YouTube links has an error, which might be a mistake when creating the hyperlink in www.mmds.org.\
The 25th and the 81th element in yt_links are the same. Replace it with the right video link.

In [2]:
yt_links[24]='https://www.youtube.com/watch?v=_YxHZ74QImo'

Download the YouTube videos from the fetched links.

In [16]:
from pytube import YouTube, Playlist

fail_list=[]
for video_url in yt_links:
    try:
        yt = YouTube(video_url)
        streams = yt.streams
        streams.get_highest_resolution().download()
    except:
        fail_list.append(video_url)

Save the YouTube video link, id, title(video file name) for later use.

In [15]:
from pytube import YouTube, Playlist

fail_list=[]
video_info={}
for video_url in yt_links:
    try:
        yt = YouTube(video_url)
        video_info[yt.title]={'link':video_url, 'id':yt.video_id}
    except:
        fail_list.append(video_url)

In [18]:
import json
with open('YouTube_video_info.json', "w") as json_file:
    json.dump(video_info, json_file) 

Use downsub.com to download the subtitles(.srt file) of YouTube videos above.

In [ ]:
from pytube import YouTube, Playlist
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()
driver.maximize_window() 

driver.get("https://downsub.com/")

for video_url in yt_links:

    input_blk = driver.find_element(By.XPATH, '/html/body/div[1]/div/main/div/div[1]/div/div[2]/form/div/div[1]/div[1]/div[1]/input')#//*[@id="input-34"]
    for i in range(10):
        input_blk.send_keys(Keys.CONTROL, "a")
        input_blk.send_keys(Keys.DELETE)
        time.sleep(0.1)
    input_blk.send_keys(video_url)
    time.sleep(2)
    
    submit_button = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div[1]/div/div[2]/form/div/div[2]/button/span')
    submit_button.click()
    time.sleep(10)

    download_button = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/button[1]/span/button/span')
    download_button.click()
    time.sleep(2)

driver.quit()